In [15]:
#shutil.rmtree('./data')


In [16]:
#import os
#import glob

#files = glob.glob('./working/')
#for f in files:
#    os.remove(f)

In [17]:
import os
os.makedirs('../working/data/train_seg/idc-minus/')     
os.makedirs('../working/data/train_seg/idc-plus/')  
os.makedirs('../working/data/test_seg/idc-minus/')     
os.makedirs('../working/data/test_seg/idc-plus/')  
os.makedirs('../working/data/val_seg/idc-minus/')     
os.makedirs('../working/data/val_seg/idc-plus/')  


In [60]:
sampling_seed=0
size_4_training=400
img_x=224
epoch_4_test=3
n_of_classes=2
#training_reshape=(-1, img_size, img_size, 3)

In [19]:
'''
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import keras
print(keras.__version__)
import tensorflow
print(keras.__version__)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
!pip install Livelossplot
from livelossplot import PlotLossesKeras

from glob import glob
import os
import shutil

import numpy as np
import pandas as pd
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
'''

'\n# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load\nimport keras\nprint(keras.__version__)\nimport tensorflow\nprint(keras.__version__)\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\n\nimport warnings\nwarnings.filterwarnings(\'ignore\')\n%matplotlib inline\n!pip install Livelossplot\nfrom livelossplot import PlotLossesKeras\n\nfrom glob import glob\nimport os\nimport shutil\n\nimport numpy as np\nimport pandas as pd\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version 

In [20]:
'''
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import scipy

#import tensorflow as tf
print(keras.__version__)
#print(tf.__version__)
from keras.applications import *
from keras.optimizers import *
from keras.losses import *
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.preprocessing.image import *
from keras.utils import *
# import pydot

from sklearn.metrics import *
from sklearn.model_selection import *
import tensorflow.keras.backend as K

from tqdm import tqdm, tqdm_notebook
from colorama import Fore
import json
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from skimage.io import *
%config Completer.use_jedi = False
import time
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import xgboost as xgb

print("All modules have been imported")
'''

'\nimport numpy as np\nimport pandas as pd\nimport cv2\nfrom PIL import Image\nimport scipy\n\n#import tensorflow as tf\nprint(keras.__version__)\n#print(tf.__version__)\nfrom keras.applications import *\nfrom keras.optimizers import *\nfrom keras.losses import *\nfrom keras.layers import *\nfrom keras.models import *\nfrom keras.callbacks import *\nfrom keras.preprocessing.image import *\nfrom keras.utils import *\n# import pydot\n\nfrom sklearn.metrics import *\nfrom sklearn.model_selection import *\nimport tensorflow.keras.backend as K\n\nfrom tqdm import tqdm, tqdm_notebook\nfrom colorama import Fore\nimport json\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom glob import glob\nfrom skimage.io import *\n%config Completer.use_jedi = False\nimport time\nfrom sklearn.decomposition import PCA\nfrom sklearn.svm import LinearSVC\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.metrics import accuracy_score\nimport lightgbm as lgb\nimport xgboost as xgb\n\n

In [61]:
import random
from sklearn.model_selection import train_test_split

import shutil
from glob import glob 
#make directory for labelling

train_dir='../working/data/train_seg/'
validation_dir='../working/data/val_seg/'
#train_dir='../working/data/train_seg/'  
#test_dir = '../working/data/test_seg/normal'  
test_dir = '../working/data/test_seg/'  

class0 = [] # 0 = idc+
class1 = [] # 1 = idc-
imagePatches = glob('../input/breast-histopathology-images/IDC_regular_ps50_idx5/**/*.png', recursive=True)
for filename in imagePatches:
    if filename.endswith("class0.png"):
         class0.append(filename)
    else:
        class1.append(filename)

print(class0[0:10])

#sampling 10000 images from class 0 and class 1 to train the model

random.seed(sampling_seed)
class0sample=random.sample(class0,size_4_training)
class0label=np.zeros(size_4_training)
class1sample=random.sample(class1,size_4_training)
class1label=np.ones(size_4_training)

class0sample_train, class0sample_test1, class0label_train, class0label_test1 = train_test_split(class0sample, class0label, test_size=0.3, random_state=42)
class0sample_test,class0sample_val,  class0label_test, class0label_val = train_test_split(class0sample_test1, class0label_test1, test_size=0.3, random_state=42)
print(len(class0sample_train))
print(len(class0sample_test))
print(len(class0sample_val))
class1sample_train, class1sample_test1, class1label_train, class1label_test1 = train_test_split(class1sample, class1label, test_size=0.3, random_state=42)
class1sample_test, class1sample_val, class1label_test, class1label_val = train_test_split(class1sample_test1, class1label_test1, test_size=0.3, random_state=42)
print(len(class1sample_train))
print(len(class1sample_test))
print(len(class1sample_val))
def read_and_save_data(path, file_name_array):
    j=0
    for i in file_name_array:
        if i.endswith('.png'):
          
            
            #second copy method
            head, tail = os.path.split(i)
            outputname=str(path+tail)
            #outputname=str(path+str(j)+'.png')
            print(outputname)
            shutil.copy(i, outputname)
   
            #print(status2)
            
            j=j+1
            if j==120000:
                break
            
   
class0train_path='../working/data/train_seg/idc-minus/'
class1train_path='../working/data/train_seg/idc-plus/'
class0test_path='../working/data/test_seg/idc-minus/'
class1test_path='../working/data/test_seg/idc-plus/'
class0val_path='../working/data/val_seg/idc-minus/'
class1val_path='../working/data/val_seg/idc-plus/'

read_and_save_data(class0train_path,class0sample_train)
read_and_save_data(class1train_path,class1sample_train)


read_and_save_data(class0test_path,class0sample_test)
read_and_save_data(class1test_path,class1sample_test)


read_and_save_data(class0val_path,class0sample_val)
read_and_save_data(class1val_path,class1sample_val)

['../input/breast-histopathology-images/IDC_regular_ps50_idx5/10295/0/10295_idx5_x1351_y1101_class0.png', '../input/breast-histopathology-images/IDC_regular_ps50_idx5/10295/0/10295_idx5_x1501_y501_class0.png', '../input/breast-histopathology-images/IDC_regular_ps50_idx5/10295/0/10295_idx5_x1501_y1101_class0.png', '../input/breast-histopathology-images/IDC_regular_ps50_idx5/10295/0/10295_idx5_x451_y901_class0.png', '../input/breast-histopathology-images/IDC_regular_ps50_idx5/10295/0/10295_idx5_x801_y451_class0.png', '../input/breast-histopathology-images/IDC_regular_ps50_idx5/10295/0/10295_idx5_x151_y1051_class0.png', '../input/breast-histopathology-images/IDC_regular_ps50_idx5/10295/0/10295_idx5_x1351_y901_class0.png', '../input/breast-histopathology-images/IDC_regular_ps50_idx5/10295/0/10295_idx5_x701_y651_class0.png', '../input/breast-histopathology-images/IDC_regular_ps50_idx5/10295/0/10295_idx5_x951_y1401_class0.png', '../input/breast-histopathology-images/IDC_regular_ps50_idx5/102

In [22]:
#shutil.rmtree('./data')

**load gen**

In [23]:

import os #Operating System
import sys #System
# train_generator = train_datagen.flow(x_train, y_train, batch_size =)
# val_generator = val_datagen.flow(x_val, y_val, batch_size = 64)
# test_generator=test_datagen.flow(x_test,y_test,batch_size = 64)

In [62]:
import os
import cv2
def readImage(path, tag):
    j=0
    image_data = []
    label=[]
    for i in os.listdir(path):
        imgname=path+i
        #print(imgname)
        img = cv2.imread(imgname, cv2.IMREAD_COLOR)
        img_resized = cv2.resize(img, (img_x,img_x), interpolation=cv2.INTER_LINEAR)
        image_data.append(img_resized)
        label.append(tag)
        #print(img[1])
        j=j+1
        #if j==10:
        #    break
        
    return image_data, label

import numpy as np
from tensorflow.keras.utils import *
from sklearn.utils import shuffle

class0_train, train0_label = readImage(class0train_path, 0)
class1_train, train1_label  = readImage(class1train_path, 1)
class0_test, test0_label = readImage(class0test_path, 0)
class1_test, test1_label = readImage(class1test_path, 1)
class0_val, val0_label = readImage(class0val_path, 0)
class1_val, val1_label = readImage(class1val_path, 1)

def Image_array_process(class0array,label0, class1array, label1):
    class0_array=np.array(class0array)
    class1_array=np.array(class1array)
    combined_data = np.concatenate((class0_array, class1_array))
    combined_label= np.concatenate((label0,label1), axis=0)
    assert len(combined_data) == len(combined_label)
    combined_data, combined_label = shuffle(combined_data, combined_label, random_state=0)
    print(combined_data.shape)
    length=len(combined_data)
    #combined_label=to_categorical(combined_label,num_classes=2)
    #combined_label=np.array(combined_label,dtype=np.int64)
    #combined_data=np.array(combined_data,dtype=np.int64)
    i=0
    for i in range(length):
        print(combined_label[i])

    print

    #print(class0_array.shape)
    #print(combined_data.shape)
    '''
    training_reshape=(img_x,img_x,3)
    length=len(combined_data)
    print(length)
    x =[None]*length
    #print(img_data.type)
    y =np.zeros(length)
    i=0
   
    for features,label in combined_data:
        x[i]=features
        #print(x.shape)
        y[i]=label
        #print(y[i])
        i=i+1
    
            #x = np.array(x).reshape(training_reshape)
    x = np.array(x)    
    #print(x.shape)
    #y=np.array(y)
    y=y.astype(int)
    y = to_categorical(y)
    print(y)
    '''  
    return combined_data, combined_label


X_train, y_train=Image_array_process(class0_train, train0_label, class1_train, train1_label)
X_test, y_test=Image_array_process(class0_test, test0_label, class1_test, test1_label)
X_val, y_val=Image_array_process(class0_val, val0_label, class1_val, val1_label)


(649, 224, 224, 3)
1
1
1
1
1
1
1
0
1
0
1
1
0
0
1
0
0
1
0
1
0
0
1
1
1
1
0
1
0
1
0
1
1
1
0
1
1
0
0
1
1
1
1
0
1
1
0
1
1
1
0
1
0
0
1
1
0
0
0
0
1
0
1
1
0
0
0
0
1
1
1
1
0
1
0
0
1
1
0
0
0
1
1
0
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0
1
1
1
1
0
1
1
0
1
1
0
0
0
1
1
0
0
1
0
1
0
1
1
1
1
0
1
0
1
1
0
0
1
1
1
1
1
1
1
0
0
0
0
0
0
1
0
1
1
1
1
0
0
0
1
1
0
0
0
1
1
0
1
0
1
0
0
1
1
1
0
1
1
1
1
1
0
1
1
1
1
0
1
0
0
0
1
1
0
0
1
1
1
1
0
1
1
1
0
1
0
1
1
0
0
1
1
1
0
1
0
1
1
0
1
0
0
0
1
0
0
1
1
1
1
1
1
1
0
1
0
1
0
0
0
0
0
1
0
0
1
1
1
0
0
1
1
1
0
1
1
1
0
1
1
1
0
1
0
1
1
1
1
1
1
1
0
0
0
0
1
1
1
1
0
1
0
1
1
0
1
0
0
0
0
1
1
1
0
0
0
1
0
1
0
1
0
1
1
0
0
1
1
1
1
1
0
0
0
0
1
1
1
1
1
1
0
0
1
1
1
1
0
1
0
0
0
1
1
1
1
0
0
1
0
1
1
0
1
0
0
0
1
0
0
1
1
0
0
1
1
1
1
1
0
1
0
1
1
1
1
0
0
0
1
1
0
0
0
0
0
1
1
1
1
1
1
0
1
0
1
1
1
0
0
0
0
1
1
1
1
0
1
1
1
0
0
1
0
1
0
0
0
1
1
0
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
0
0
0
0
0
1
0
0
1
1
0
1
0
1
1
1
1
1
1
1
1
0
1
1
1
0
0
1
0
0
1
0
1
1
1
1
1
1
0
0
0
1
0
0
1
1
0
0
0
1
1
1
0
1
0
0
1
0
0
0
1
0
0

In [29]:
train_data=X_train
train_label=y_train

test_data=X_test
test_label=y_test

val_data=X_val
val_label=y_val

In [ ]:
'''

train_dir='../working/data/train_seg/'  
#test_dir = '../working/data/test_seg/idc-minus'  
test_minus_dir = '../working/data/test_seg/idc-minus' 
img_size = (img_x, img_x)  
epochs = epoch_4_test
MODEL_PATH = '../working/log/tst_model.h5'
board_name1 = '../working/log/stage1/' + now + '/'
board_name2 = '../working/log/stage2/' + now + '/'

#MODEL_PATH = '../working/log/tst_model.h5'
#model.load_weights(MODEL_PATH)
# --------------- test ----------------
dirs = os.listdir(test_minus_dir)

prob_list = []
for d in dirs:
    img = image.load_img(test_minus_dir + os.sep + d, target_size=img_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    y = model.predict(x)
#    print(y)
    #print(classes[np.argmax(y)])
#    print(classes)

    prob_list.append(str(y))   
    file=open('../working/pred_prob.txt','w') 
    file.write('\n'.join(prob_list))
    file.close()
    
#import tensorflow as tf
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#keras.backend.set_session(tf.Session(config=config))
'''

PLTv3

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jun 16 22:23:24 2020

@author: HP
"""
'''

from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate, Conv2D, Add, Activation, Lambda
from keras import backend as K
from keras.activations import sigmoid


from keras import backend as K
from keras import optimizers
from tensorflow.keras.optimizers import Adam
import keras.backend.tensorflow_backend as KTF
#import tensorflow.python.keras.backend as KTF
import glob
from keras.layers import Input,Dense,Dropout,BatchNormalization,Conv2D,MaxPooling2D,AveragePooling2D,concatenate,Activation,ZeroPadding2D
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd
import keras
from keras.models import load_model
from keras.layers import Activation, Dense
from matplotlib import pyplot as plt
from skimage import io,data
import time
from keras import layers
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.preprocessing.image import ImageDataGenerator

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
now = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())

import os,sys
os.getcwd()
#os.chdir("../working/cjd/31_CNN_Attention")

#import tensorflow as tf
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras import regularizers
'''

In [3]:
import tensorflow as tf
from keras import backend as K
from keras import optimizers
from tensorflow.keras.optimizers import Adam
#import keras.backend.tensorflow_backend as KTF
from tensorflow.python.keras.backend import get_session
import glob
from keras.layers import Input,Dense,Dropout,BatchNormalization,Conv2D,MaxPooling2D,AveragePooling2D,concatenate,Activation,ZeroPadding2D
import cv2
import numpy as np
import pandas as pd
import keras as kr
from keras.models import load_model
from keras.layers import Activation, Dense
from matplotlib import pyplot as plt
from skimage import io,data
import time
#from keras import layers
#from keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow import keras 
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier



import os,sys
#os.getcwd()
#os.chdir("/home/cjd/28_Entity_train")
print(os.getcwd())
print (sys.version)


/kaggle/working
3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]


In [4]:
now = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
#import os
# 
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3,5"

'''
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

'''

'\nconfig = tf.ConfigProto()\nconfig.gpu_options.allow_growth = True\nkeras.backend.tensorflow_backend.set_session(tf.Session(config=config))\n\n'

In [7]:

batch_size = 32 #每批训练数据量的大小，批量多设一些；GoogleNet批量值设为20 2^n
epochs = epoch_4_test
MODEL_INIT = './init_model.h5'
MODEL_PATH = './test_model.h5'
board_name1 = './obj_reco/stage1/' + now + '/'
board_name2 = './obj_reco/stage2/' + now + '/'
train_dir='/data/train_seg/'
validation_dir='/data/test_seg/'
img_size = (img_x, img_x)  # 图片大小
#classes=list(range(1,5))
#classes=['1','2','3','4']
nb_train_samples = len(glob.glob(train_dir + '/*/*.*'))  # 训练样本数
nb_validation_samples = len(glob.glob(validation_dir + '/*/*.*'))  # 验证样本数

#classes = sorted([o for o in os.listdir(train_dir)])  # 根据文件名分类

In [63]:


import tensorflow as tf        
def focal_loss(gamma=2.):            
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        return -K.sum( K.pow(1. - pt_1, gamma) * K.log(pt_1)) 
    return focal_loss_fixed


def Conv2d_BN(x, nb_filter,kernel_size, strides=(1,1), padding='same',name=None):  
    if name is not None:  
        bn_name = name + '_bn'  
        conv_name = name + '_conv'  
    else:  
        bn_name = None  
        conv_name = None  
  
    x = Conv2D(nb_filter,kernel_size,padding=padding,strides=strides,activation='relu',name=conv_name)(x)  
    x = BatchNormalization(axis=3,name=bn_name)(x)  
    return x  

def Conv_Block(inpt,nb_filter,kernel_size,strides=(1,1), with_conv_shortcut=False):  
    x = Conv2d_BN(inpt,nb_filter=nb_filter[0],kernel_size=(1,1),strides=strides,padding='same')  
    x = Conv2d_BN(x, nb_filter=nb_filter[1], kernel_size=(3,3), padding='same')  
    x = Conv2d_BN(x, nb_filter=nb_filter[2], kernel_size=(1,1), padding='same')  
    if with_conv_shortcut:  
        shortcut = Conv2d_BN(inpt,nb_filter=nb_filter[2],strides=strides,kernel_size=kernel_size)  
        x = add([x,shortcut])  
        return x  
    else:  
        x = add([x,inpt])  
        return x  



class SeBlock(keras.layers.Layer):   
    def __init__(self, reduction=4,**kwargs):
        super(SeBlock,self).__init__(**kwargs)
        self.reduction = reduction
    def build(self,input_shape):
    	#input_shape     
    	pass
    def call(self, inputs):
        x = keras.layers.GlobalAveragePooling2D()(inputs)
        x = keras.layers.Dense(int(x.shape[-1]) // self.reduction, use_bias=False,activation=keras.activations.relu)(x)
        x = keras.layers.Dense(int(inputs.shape[-1]), use_bias=False,activation=keras.activations.hard_sigmoid)(x)
        return keras.layers.Multiply()([inputs,x])    
        #return inputs*x 

#------#MS-DenseNet----------------------------------
def get_bottlenet(image_size,alpha=1.0):
    inputs = keras.layers.Input(shape=(image_size,image_size,3),name='input_1')
    net = keras.layers.ZeroPadding2D(padding=(3,3),name='zero_padding2d_1')(inputs)
    net = keras.layers.Conv2D(filters=64, kernel_size=(7,7), strides=(2,2),
                             padding='valid', name='conv1/conv')(net)
    net = keras.layers.BatchNormalization(name='conv1/bn')(net)
    net = keras.layers.ReLU(name='conv1/relu')(net)
    net = keras.layers.ZeroPadding2D(padding=(1,1),name='zero_padding2d_2')(net)
#    se = keras.layers.GlobalAveragePooling2D(name='transform2_pool')(net)
    se=SeBlock()(net)

    net = keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid',
                                   name='pool1')(net)
#    se = keras.layers.GlobalAveragePooling2D(name='transform2_pool')(net)
    se=SeBlock()(net)
    
    for i in range(int(3*alpha)):
        block = net
        block = keras.layers.SeparableConv2D(filters=32,kernel_size=(3,3), strides=(1, 1), 
                                    padding='same',
                                    name='conv2_block{}_2_sepconv'.format(i))(block)
        net = keras.layers.Concatenate(name='conv2_block{}_concat'.format(i))([net,block])    
    net = keras.layers.BatchNormalization(name='pool2_bn')(net) 
    net = keras.layers.ReLU(name='pool2_relu')(net)
    eq = keras.layers.Dense(units=net.shape[-1],activation='sigmoid',
                            name='transform2_dense1')(se)
    net = keras.layers.Multiply(name='transform2_multiply')([net,eq])

    net = keras.layers.Conv2D(filters=int(net.shape[-1])//2,kernel_size=(1,1),strides=(1,1),
                              padding='same',name='pool2_conv')(net)
    net = keras.layers.AveragePooling2D(pool_size=(2,2),strides=(2,2),
                                        name='pool2_pool')(net)
#    se = keras.layers.GlobalAveragePooling2D(name='transform3_pool')(net)
    se=SeBlock()(net)

    for i in range(int(6*alpha)):
        block = net
        block = keras.layers.SeparableConv2D(filters=32, kernel_size=(3,3), strides=(1, 1),
                                             padding='same',
                                             name='conv3_block{}_2_sepconv'.format(i))(block)
        net = keras.layers.Concatenate(name='conv3_block{}_concat'.format(i))([net,block])
    net = keras.layers.BatchNormalization(name='pool3_bn')(net) 
    net = keras.layers.ReLU(name='pool3_relu')(net)
    eq = keras.layers.Dense(units=net.shape[-1],activation='sigmoid',
                            name='transform3_dense1')(se)
    net = keras.layers.Multiply(name='transform3_multiply')([net,eq])

    net = keras.layers.Conv2D(filters=int(net.shape[-1])//2,kernel_size=(1,1),strides=(1,1),
                              padding='same',name='pool3_conv')(net)
    net = keras.layers.AveragePooling2D(pool_size=(2,2),strides=(2,2),
                                    name='pool3_pool')(net)
#    se = keras.layers.GlobalAveragePooling2D(name='transform4_pool')(net)
    se=SeBlock()(net)
    for i in range(int(12*alpha)):
        block = net
        block = keras.layers.SeparableConv2D(filters=32, kernel_size=(3,3), strides=(1, 1), 
                                    padding='same',
                                    name='conv4_block{}_2_sepconv'.format(i))(block)
        net = keras.layers.Concatenate(name='conv4_block{}_concat'.format(i))([net,block])
    net = keras.layers.BatchNormalization(name='pool4_bn')(net) 
    net = keras.layers.ReLU(name='pool4_relu')(net)
    eq = keras.layers.Dense(units=net.shape[-1],activation='sigmoid',
                            name='transform4_dense1')(se)
    net = keras.layers.Multiply(name='transform4_multiply')([net,eq])

    net = keras.layers.Conv2D(filters=int(net.shape[-1])//2,kernel_size=(1,1),strides=(1,1),
                             padding='same',name='pool4_conv')(net)
    net = keras.layers.AveragePooling2D(pool_size=(2,2),strides=(2,2),
                                        name='pool4_pool')(net)
#    se = keras.layers.GlobalAveragePooling2D(name='transform5_pool')(net)
    se=SeBlock()(net)
    for i in range(int(8*alpha)):
        block = net
        block = keras.layers.SeparableConv2D(filters=32, kernel_size=(3,3), strides=(1, 1), 
                                    padding='same',
                                    name='conv5_block{}_2_sepconv'.format(i))(block)
        net = keras.layers.Concatenate(name='conv5_block{}_concat'.format(i))([net,block])
       
    eq = keras.layers.Dense(units=net.shape[-1],activation='sigmoid',
                            name='transform5_dense1')(se)
    net = keras.layers.Multiply(name='transform5_multiply')([net,eq])
    net = keras.layers.BatchNormalization(name='bn')(net)
    net = keras.layers.ReLU(name='relu')(net)
    model = keras.Model(inputs=inputs,outputs=net,name='mobile_densenet_bottle')
    return model

def get_model(image_size, alpha, classes):
    bottlenet = get_bottlenet(alpha=alpha,image_size=image_size)
    net = keras.layers.GlobalAveragePooling2D(name='global_pool')(bottlenet.output)
    net = keras.layers.Dropout(rate=0.4,name='dropout1')(net)
    net = keras.layers.Dropout(rate=0.4,name='dropout2')(net)
    output = keras.layers.Dense(units=classes,activation='softmax',
                             name='prediction')(net)
    model = keras.Model(inputs=bottlenet.input,outputs=output,name='mobile_densenet')
    return model
model = get_model(image_size=img_x, alpha=1.0, classes=n_of_classes)
model.summary()

Model: "mobile_densenet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________

In [56]:
#train_datagen = ImageDataGenerator(validation_split=0.2)
train_datagen = ImageDataGenerator()

train_datagen.mean = np.array([103.939, 116.779, 123.68], dtype=np.int64).reshape((3, 1, 1))  # remove imagenet BGR mean value
#train_generator = train_datagen.flow(train_data, train_label, target_size=img_size, classes=classes)
train_generator = train_datagen.flow(train_data, train_label)

validation_datagen = ImageDataGenerator()
validation_datagen.mean = np.array([103.939, 116.779, 123.68], dtype=np.int64).reshape((3, 1, 1))
test_dir = '../working/data/test_seg/'  
#val_generator = validation_datagen.flow(test_data, test_label, target_size=img_size, classes=classes)
val_generator = validation_datagen.flow(test_data, test_label)

val2_datagen = ImageDataGenerator()
val2_datagen.mean = np.array([103.939, 116.779, 123.68], dtype=np.int64).reshape((3, 1, 1))
test_dir = '../working/data/test_seg/'  
#val2_generator = val2_datagen.flow(val_data, val_label, target_size=img_size, classes=classes)
val2_generator = val2_datagen.flow(val_data, val_label)

In [53]:
print(val2_generator)

In [81]:
tf.config.run_functions_eagerly(True)

In [82]:
import glob
IMG_SHAPE=(img_x, img_x, 3)
model_checkpoint2 = ModelCheckpoint(filepath=MODEL_PATH,  monitor='val_loss', verbose=1, save_best_only=True, mode='min')
board2 = TensorBoard(log_dir=board_name2,
                     histogram_freq=0,
                     write_graph=True,
                     write_images=True)
callback_list2 = [model_checkpoint2, board2]
learning_rate = 0.01
decay = 1e-6
momentum = 0.8
nesterov = True
sgd_optimizer = keras.optimizers.SGD(lr = learning_rate, decay = decay,            
                    momentum = momentum, nesterov = nesterov)
model.compile(loss = [focal_loss(gamma=2)], optimizer = sgd_optimizer, metrics = ['acc'])

nb_train_samples = len(glob.glob(train_dir + '/*/*.*'))  
nb_validation_samples = len(glob.glob(validation_dir + '/*/*.*'))  

model.fit_generator(train_generator, steps_per_epoch=nb_train_samples //batch_size, epochs=epochs,
                    validation_data=val_generator, validation_steps=nb_validation_samples // batch_size,
                    callbacks=callback_list2, verbose=1)




2022-05-14 18:57:54.328677: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-05-14 18:57:54.328818: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-05-14 18:57:54.330036: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/3
 1/20 [>.............................] - ETA: 1:07 - loss: 12.2920 - acc: 0.5938

2022-05-14 18:57:57.953787: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-05-14 18:57:57.953841: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


 2/20 [==>...........................] - ETA: 49s - loss: 29.2822 - acc: 0.5781 

2022-05-14 18:58:00.678870: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-05-14 18:58:00.711877: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-05-14 18:58:00.755749: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./obj_reco/stage2/2022-05-14_17-56-00/train/plugins/profile/2022_05_14_18_58_00

2022-05-14 18:58:00.781756: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ./obj_reco/stage2/2022-05-14_17-56-00/train/plugins/profile/2022_05_14_18_58_00/6c9e4329e324.trace.json.gz
2022-05-14 18:58:00.830775: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./obj_reco/stage2/2022-05-14_17-56-00/train/plugins/profile/2022_05_14_18_58_00

2022-05-14 18:58:00.836887: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to ./obj_reco/st

20/20 [==============================] - 14s 557ms/step - loss: 94.1924 - acc: 0.5286 - val_loss: inf - val_acc: 0.5000

Epoch 00001: val_loss did not improve from inf


In [ ]:
#model.fit(
#    train_generator,
#    batch_size=batch_size,
#    epochs=epoch_4_test,
#    verbose=1,
#    callbacks=callback_list2,
#    validation_data=val_generator,
#    shuffle=True,
#    class_weight=None,
#    sample_weight=None,
#   initial_epoch=0,
#   steps_per_epoch=None,
#   validation_steps=None,
#   validation_batch_size=None,
#   validation_freq=1,
#   max_queue_size=10,
#   workers=1,
#   use_multiprocessing=False
#)

In [74]:
import glob
nb_train_samples = len(glob.glob(train_dir + '/*/*.*'))  
nb_validation_samples = len(glob.glob(validation_dir + '/*/*.*'))  
print(nb_train_samples //batch_size)
print(nb_validation_samples //batch_size)

20
2


In [66]:
print(val_generator.next())

(array([[[[187., 160., 221.],
         [187., 160., 221.],
         [188., 162., 222.],
         ...,
         [228., 214., 233.],
         [228., 214., 233.],
         [228., 214., 233.]],

        [[187., 160., 221.],
         [187., 160., 221.],
         [188., 162., 222.],
         ...,
         [228., 214., 233.],
         [228., 214., 233.],
         [228., 214., 233.]],

        [[188., 161., 221.],
         [188., 161., 221.],
         [189., 163., 222.],
         ...,
         [228., 214., 233.],
         [228., 214., 233.],
         [228., 214., 233.]],

        ...,

        [[242., 241., 244.],
         [242., 241., 244.],
         [241., 241., 243.],
         ...,
         [207., 184., 222.],
         [207., 183., 222.],
         [207., 183., 222.]],

        [[242., 241., 244.],
         [242., 241., 244.],
         [242., 240., 244.],
         ...,
         [210., 187., 225.],
         [209., 186., 225.],
         [209., 186., 225.]],

        [[242., 241., 244.],
      

In [35]:
ls = tf.keras.losses.categorical_crossentropy()
ls(y_true, train_label)

TypeError: categorical_crossentropy() missing 2 required positional arguments: 'y_true' and 'y_pred'

In [ ]:
'''
#---------------1-st stage---------------------------------------------
#model_checkpoint2 = ModelCheckpoint(filepath=MODEL_PATH,  monitor='val_accuracy')
#model_checkpoint2 = ModelCheckpoint('tst_model.h5',  monitor='val_accuracy', save_best_only=True, mode='max')
#board2 = TensorBoard(log_dir=board_name2, histogram_freq=0, write_graph=True, write_images=True)
#callback_list2 = [model_checkpoint2, board2]
#callbacks2 = [EarlyStopping(monitor='val_loss', patience=5, verbose=2), ModelCheckpoint('test_model.hdf5', save_best_only=True), TensorBoard(log_dir=board_name2,
#                     histogram_freq=0,
#                     write_graph=True,
#                     write_images=True)]
callbacks1 = [ModelCheckpoint('init_model1.h5', verbose=1, save_best_only=True, mode='min'), TensorBoard(log_dir=board_name2,
                     histogram_freq=0,
                     write_graph=True,
                     write_images=True)]

#model.load_weights(MODEL_INIT)
for model1 in model.layers:
    model1.trainable = True
#fine_tune_at = 50
#for layer in model.layers[:fine_tune_at]:
#    layer.trainable = False


#model.compile(optimizer=optimizers.Adam(), loss =[focal_loss(gamma=2)], metrics=['accuracy']) #loss='categorical_crossentropy',
model.compile(optimizer=optimizers.SGD(lr=0.0001), loss = [focal_loss(gamma=2)], metrics=['accuracy']) #loss='categorical_crossentropy',
#model.compile(optimizer=optimizers.Adadelta(), loss = [focal_loss(gamma=2)], metrics=['accuracy']) #loss='categorical_crossentropy',

history1=model.fit_generator(train_generator, steps_per_epoch=nb_train_samples / float(batch_size), epochs=epochs, validation_data=val_generator, validation_steps=nb_validation_samples / float(batch_size), callbacks=callbacks1, verbose=2)



from contextlib import redirect_stdout   
with open('./model_summary.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary(line_length=200,positions=[0.30,0.60,0.7,1.0])

        '''





In [ ]:
'''
#-------------2nd stage------------------------------------------------------------
history2=model.fit_generator(train_generator, steps_per_epoch=nb_train_samples / float(batch_size),
                           epochs = epochs,
                           validation_steps=nb_validation_samples / float(batch_size),
                           validation_data=val_generator,
                           callbacks=callbacks2, verbose=1)
'''

In [ ]:
#model.load_weights('./init_model.h5')

In [ ]:
'''
#---------------3-rd stage---------------------------------------------
#model_checkpoint2 = ModelCheckpoint(filepath=MODEL_PATH,  monitor='val_accuracy')
#model_checkpoint2 = ModelCheckpoint('tst_model.h5',  monitor='val_accuracy', save_best_only=True, mode='max')
#board2 = TensorBoard(log_dir=board_name2, histogram_freq=0, write_graph=True, write_images=True)
#callback_list2 = [model_checkpoint2, board2]
#callbacks2 = [EarlyStopping(monitor='val_loss', patience=5, verbose=2), ModelCheckpoint('test_model.hdf5', save_best_only=True), TensorBoard(log_dir=board_name2,
#                     histogram_freq=0,
#                     write_graph=True,
#                     write_images=True)]
callbacks3 = [ModelCheckpoint('test_model.h5', verbose=1, save_best_only=True, mode='min'), TensorBoard(log_dir=board_name2,
                     histogram_freq=0,
                     write_graph=True,
                     write_images=True)]

#model.load_weights(MODEL_INIT)
for model1 in model.layers:
    model1.trainable = True
#fine_tune_at = 50
#for layer in model.layers[:fine_tune_at]:
#    layer.trainable = False


#model.compile(optimizer=optimizers.Adam(), loss =[focal_loss(gamma=2)], metrics=['accuracy']) #loss='categorical_crossentropy',
model.compile(optimizer=optimizers.SGD(lr=0.0001), loss = [focal_loss(gamma=2)], metrics=['accuracy']) #loss='categorical_crossentropy',
#model.compile(optimizer=optimizers.Adadelta(), loss = [focal_loss(gamma=2)], metrics=['accuracy']) #loss='categorical_crossentropy',

history3=model.fit_generator(train_generator, steps_per_epoch=nb_train_samples / float(batch_size), epochs=epochs, validation_data=val_generator, validation_steps=nb_validation_samples / float(batch_size), callbacks=callbacks3, verbose=1)



from contextlib import redirect_stdout   
with open('./model_summary.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary(line_length=200,positions=[0.30,0.60,0.7,1.0])

'''




In [ ]:

import seaborn as sns
from sklearn.metrics import f1_score, roc_auc_score, cohen_kappa_score, precision_score, recall_score, accuracy_score, confusion_matrix

def test_2_final_model(model, train_generator, test_generator, val_generator, y_train, y_test, y_val, class_labels):
    
    # BS = 16
    ##results = dict()
    
    # n = len(testy)// BS

    # testX = testX[:BS*n]
    # testy = testy[:BS*n]

    ##print('Predicting test data')
    ##test_start_time = datetime.now()
    y_pred_test_original = model.predict_generator(test_generator,verbose=1)
    # y_pred = (y_pred_test>0.5).astype('int')

    y_pred_test = np.argmax(y_pred_test_original, axis = 1)
 
    y_test = y_test.astype(int) # sparse form not categorical
    y_train = y_train.astype(int) # sparse form not categorical
    y_val = y_val.astype(int) # sparse form not categorical
    # y_test = np.argmax(testy, axis= 1)
    #y_test = np.argmax(testy, axis=-1)
    
    ##test_end_time = datetime.now()
    ##print('Done \n \n')
    ##results['testing_time'] = test_end_time - test_start_time
    ##print('testing time(HH:MM:SS:ms) - {}\n\n'.format(results['testing_time']))
    ##results['predicted'] = y_pred_test


    # balanced_accuracy
    from sklearn.metrics import balanced_accuracy_score
    ##balanced_accuracy = balanced_accuracy_score(y_true=y_test, y_pred=y_pred_test)
    ##print('---------------------')
    ##print('| Balanced Accuracy  |')
    ##print('---------------------')
    ##print('\n    {}\n\n'.format(balanced_accuracy))

    
    # calculate overall accuracty of the model
    ##accuracy = metrics.accuracy_score(y_true=y_test, y_pred=y_pred_test)
    # store accuracy in results
    ##results['accuracy'] = accuracy
    ##print('---------------------')
    ##print('|      Accuracy      |')
    ##print('---------------------')
    ##print('\n    {}\n\n'.format(accuracy))
    

    # get classification report
    ##print('-------------------------')
    ##print('| Classifiction Report |')
    ##print('-------------------------')
    ##classification_report = metrics.classification_report(y_test, y_pred_test)
    # store report in results
    ##results['classification_report'] = classification_report
    ##print(classification_report)
    
    
    
    # confusion matrix
    ##cm = metrics.confusion_matrix(y_test, y_pred_test)
    ##results['confusion_matrix'] = cm
    ##if print_cm: 
    ##    print('--------------------')
    ##    print('| Confusion Matrix |')
    ##    print('--------------------')
    ##    print('\n {}'.format(cm))
        
    # plot confusin matrix
    ##plt.figure(figsize=(6,4))
    ##plt.grid(b=False)
    ##plot_confusion_matrix(cm, classes=class_labels, normalize=True, title='Normalized confusion matrix')
    ##plt.show()
    

    
    # add the trained  model to the results
    ##results['model'] = model
    ##print("calculate other score")
    ##print("predicting validation data")

    #calculate other score
    y_pred_val_original = model.predict_generator(val_generator,verbose=1)
    # y_pred = (y_pred_val>0.5).astype('int')

    y_pred_val = np.argmax(y_pred_val_original, axis = 1)
    # y_val = np.argmax(valy, axis= 1)
    #y_val = np.argmax(valy, axis=-1)
    print("predicting test data")
    y_pred_train_original = model.predict_generator(train_generator,verbose=1)
    # y_pred = (y_pred_train>0.5).astype('int')

    y_pred_train = np.argmax(y_pred_train_original, axis = 1)
    # y_train = np.argmax(trainy, axis= 1)
    #y_train = np.argmax(trainy, axis=-1)
    
    print("Train accuracy Score------------>")
    print ("{0:.3f}".format(accuracy_score(y_train, y_pred_train)*100), "%")
    
    print("Val accuracy Score--------->")
    
    print("{0:.3f}".format(accuracy_score(y_val, y_pred_val)*100), "%")
    

    
  
    print("Test accuracy Score--------->")
    print("{0:.3f}".format(accuracy_score(y_test, y_pred_test)*100), "%")
    
    print("F1 Score--------------->")
    print("{0:.3f}".format(f1_score(y_test, y_pred_test, average = 'weighted')*100), "%")
    
    print("Cohen Kappa Score------------->")
    print("{0:.3f}".format(cohen_kappa_score(y_test, y_pred_test)*100), "%")
    
    print("Recall-------------->")
    print("{0:.3f}".format(recall_score(y_test, y_pred_test, average = 'weighted')*100), "%")
    
    print("Precision-------------->")
    print("{0:.3f}".format(precision_score(y_test, y_pred_test, average = 'weighted')*100), "%")
    
    cf_matrix_test = confusion_matrix(y_test, y_pred_test)
    cf_matrix_val = confusion_matrix(y_val, y_pred_val)
    
    plt.figure(figsize = (12, 6))
    plt.subplot(121)
    sns.heatmap(cf_matrix_val, annot=True, cmap='Blues')
    plt.title("Val Confusion matrix")
    
    plt.subplot(122)
    sns.heatmap(cf_matrix_test, annot=True, cmap='Blues')
    plt.title("Test Confusion matrix")
    
    plt.show()
    
    
    return

In [ ]:
#MODEL_PATH = './test_model.hdf5'
from keras.models import load_model
#best_model = load_model('./test_model.h5')
best_model = load_model('./test_model.h5', compile=False)
#best_model = load_model("./test_model.h5", custom_objects={'focal_loss_fixed': focal_loss()})

In [ ]:
test_2_final_model(best_model, train_generator, 
               val_generator, val2_generator, 
               np.argmax(train_label, axis=1),
               np.argmax(test_label, axis=1),
               np.argmax(val_label, axis=1), 
               class_labels = ['idc-', 'idc+'])

In [ ]:
'''
#MODEL_PATH = './test_model.hdf5'
from keras.models import load_model
#best_model = load_model('./init_model.h5')
init_model = load_model('./init_model.h5',compile=False)
#init_model = load_model("./init_model.h5", custom_objects={'focal_loss_fixed': focal_loss()})
'''

In [ ]:
'''
test_2_final_model(init_model, train_generator, 
               val_generator, val2_generator, 
               np.argmax(train_label, axis=1),
               np.argmax(test_label, axis=1),
               np.argmax(val_label, axis=1), 
               class_labels = ['idc-', 'idc+'])
'''

In [ ]:
model.metrics_names

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (12, 6))
plt.subplot(121)
#plotting the Accuracy of test and training sets
plt.plot(history2.history['acc'])
plt.plot(history2.history['val_acc'])
plt.title('1st training Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(122)
#plotting the loss of test and training sets
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('1st training Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
'''
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (12, 6))
plt.subplot(121)
#plotting the Accuracy of test and training sets
plt.plot(history3.history['acc'])
plt.plot(history3.history['val_acc'])
plt.title('1st training Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(122)
#plotting the loss of test and training sets
plt.plot(history3.history['loss'])
plt.plot(history3.history['val_loss'])
plt.title('1st training Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
'''